In [ ]:
#include<iostream>
#include<queue>
#include<stdlib.h>
#include<omp.h>

using namespace std;

class node {
public:
    node *left;
    node *right;
    int data;
};

class BFS {
public:
    node* insert(node*, int);
    void BFS_Function(node*);
};

node* BFS::insert(node* root, int data)
{
    if (root == NULL) {
        root = new node;
        root->data = data;
        root->left = root->right = NULL;
        return root;
    }

    if (data < root->data)
        root->left = insert(root->left, data);
    else
        root->right = insert(root->right, data);
    return root;
}

void BFS::BFS_Function(node* head)
{
    if (head == NULL) return;

    queue<node*> q;
    q.push(head);

    while (!q.empty())
    {
        int qsize = q.size();
        #pragma omp parallel for
        for (int i = 0; i < qsize; i++)
        {
            node* currNode;

            #pragma omp critical
            {
                currNode = q.front();
                q.pop();
                cout << "\t" << currNode->data;
            }

            #pragma omp critical
            {
                if (currNode->left) {
                    q.push(currNode->left);
                }
                if (currNode->right) {
                    q.push(currNode->right);
                }
            }
        }
    }
}

int main()
{
    BFS tree;
    node* root = NULL;
    int num, n;

    cout << "How many Nodes: ";
    cin >> n;

    for (int i = 0; i < n; i++)
    {
        cout << "Enter the node " << (i + 1) << ": ";
        cin >> num;

        root = tree.insert(root, num);
    }

    double start = omp_get_wtime();
    tree.BFS_Function(root);
    double end = omp_get_wtime();

    cout << "\nExecution Time: " << (end - start) << " seconds\n";
}



In [ ]:
#include<iostream>
#include<stack>
#include<stdlib.h>
#include<omp.h>

using namespace std;

class node {
public:
    node *left;
    node *right;
    int data;
};

class DFS {
public:
    node* insert(node*, int);
    void DFS_Function(node*);
};

node* DFS::insert(node* root, int data)
{
    if (root == NULL) {
        root = new node;
        root->data = data;
        root->left = root->right = NULL;
        return root;
    }

    if (data < root->data)
        root->left = insert(root->left, data);
    else
        root->right = insert(root->right, data);
    return root;
}

void DFS::DFS_Function(node* head)
{
    if (head == NULL) return;

    stack<node*> s;
    s.push(head);

    while (!s.empty())
    {
        node* currNode;

        #pragma omp parallel
        {
            #pragma omp single
            {
                currNode = s.top();
                s.pop();
                cout << "\t" << currNode->data;

                #pragma omp task
                {
                    if (currNode->right) {
                        s.push(currNode->right);
                    }
                }

                #pragma omp task
                {
                    if (currNode->left) {
                        s.push(currNode->left);
                    }
                }
            }
        }
    }
}

int main()
{
    DFS tree;
    node* root = NULL;
    int num, n;

    cout << "How many Nodes: ";
    cin >> n;

    for (int i = 0; i < n; i++)
    {
        cout << "Enter the code " << (i + 1) << ": ";
        cin >> num;

        root = tree.insert(root, num);
    }

    double start = omp_get_wtime();
    tree.DFS_Function(root);
    double end = omp_get_wtime();

    cout << "\nExecution Time: " << (end - start) << " seconds\n";
}



In [ ]:
#include<iostream>
#include<stdlib.h>
#include<omp.h>
#include<ctime>
using namespace std;

void swap(int &a, int &b)
{
    int temp = a;
    a = b;
    b = temp;
}

void bubble(int *a, int n)
{
    for (int i = 0; i < n; i++)
    {
        int first = i % 2;
        #pragma omp parallel for
        for (int j = first; j < n - 1; j += 2)
        {
            if (a[j] > a[j + 1])
            {
                swap(a[j], a[j + 1]);
            }
        }
    }
}

int main()
{
    int *a, *b, n = 10000;

    a = new int[n];
    b = new int[n];

    for (int i = 0; i < n; i++)
    {
        a[i] = rand() % 10000;
        b[i] = a[i];
    }

    double start = omp_get_wtime();
    for (int i = 0; i < n; i++)
    {
        for (int j = 0; j < n - 1; j++)
        {
            if (b[j] > b[j + 1])
            {
                swap(b[j], b[j + 1]);
            }
        }
    }
    double end = omp_get_wtime();

    double start1 = omp_get_wtime();
    bubble(a, n);
    double end1 = omp_get_wtime();


    cout << "\nSorted Array (First 100 elements):\n";
    for (int i = 0; i < 100; i++)
    {
        cout << a[i] << "\t";
        if ((i + 1) % 10 == 0)
            cout << endl;
    }

    cout << "\nSequential time: " << (end - start) << " seconds" << endl;
    cout << "Parallel time: " << (end1 - start1) << " seconds" << endl;

    delete[] a;
    delete[] b;

    return 0;
}



In [ ]:
#include<iostream>
#include<stdlib.h>
#include<omp.h>

using namespace std;

void marge(int a[], int i1, int j1, int i2, int j2)
{
  int i = i1;
  int j = i2;
  int temp[1000000];
  int k = 0;

  // Merge the two halves into temp[]
  while (i <= j1 && j <= j2)
  {
    if (a[i] < a[j])
    {
      temp[k++] = a[i++];
    }
    else
    {
      temp[k++] = a[j++];
    }
  }

  while (i <= j1)
  {
    temp[k++] = a[i++];
  }


  while (j <= j2)
  {
    temp[k++] = a[j++];
  }

  for (i = i1, j = 0; i <= j2; i++, j++)
  {
    a[i] = temp[j];
  }
}

void margesort(int a[], int i, int j)
{
  int mid;
  if (i < j)
  {
    mid = (i + j) / 2;

    #pragma omp parallel sections
    {
      #pragma omp section
      margesort(a, i, mid);

      #pragma omp section
      margesort(a, mid + 1, j);
    }

    marge(a, i, mid, mid + 1, j);
  }
}

void margeSort_seq(int a[], int i, int j)
{
  int mid;
  if (i < j)
  {
    mid = (i + j) / 2;

    margeSort_seq(a, i, mid);
    margeSort_seq(a, mid + 1, j);

    marge(a, i, mid, mid + 1, j);
  }
}

int main()
{
  int *a, *b, n = 100000;

  a = new int[n];
  b = new int[n];

  for (int i = 0; i < n; i++)
  {
    a[i] = rand() % 100000;
    b[i] = a[i];
  }


  double start = omp_get_wtime();
  margesort(a, 0, n - 1);
  double end = omp_get_wtime();

  double start1 = omp_get_wtime();
  margeSort_seq(b, 0, n - 1);
  double end1 = omp_get_wtime();

   cout << "Sorted array (Parallel Merge Sort):" << endl;
  for (int i = 0; i < 100; i++)
    {
        cout << a[i] << "\t";
        if ((i + 1) % 10 == 0)
            cout << endl;
    }

  cout << endl;

  cout << "Time for Parallel Merge Sort: " << (end - start) << " seconds" << endl;
  cout << "Time for Sequential Merge Sort: " << (end1 - start1) << " seconds" << endl;

  delete[] a;
  delete[] b;

  return 0;
}



In [ ]:
#include <iostream>
//#include <vector>
#include <omp.h>
#include <climits>
using namespace std;
void min_reduction(int arr[], int n) {
  int min_value = INT_MAX;
  #pragma omp parallel for reduction(min: min_value)
  for (int i = 0; i < n; i++) {
	if (arr[i] < min_value) {
  	min_value = arr[i];
	}
  }
  cout << "Minimum value: " << min_value << endl;
}

void max_reduction(int arr[], int n) {
  int max_value = INT_MIN;
  #pragma omp parallel for reduction(max: max_value)
  for (int i = 0; i < n; i++) {
	if (arr[i] > max_value) {
  	max_value = arr[i];
	}
  }
  cout << "Maximum value: " << max_value << endl;
}

void sum_reduction(int arr[], int n) {
  int sum = 0;
   #pragma omp parallel for reduction(+: sum)
   for (int i = 0; i < n; i++) {
	sum += arr[i];
  }
  cout << "Sum: " << sum << endl;
}

void average_reduction(int arr[], int n) {
  int sum = 0;
  #pragma omp parallel for reduction(+: sum)
  for (int i = 0; i < n; i++) {
    sum += arr[i];
  }
  cout << "Average: " << (double)sum / n << endl;
}

int main() {
    int *arr,n;
    cout<<"\n enter total no of elements=>";
    cin>>n;
    arr=new int[n];
    cout<<"\n enter elements=>";
    for(int i=0;i<n;i++)
    {
   	 cin>>arr[i];
    }

//   int arr[] = {5, 2, 9, 1, 7, 6, 8, 3, 4};
//   int n = size(arr);

  min_reduction(arr, n);
  max_reduction(arr, n);
  sum_reduction(arr, n);
  average_reduction(arr, n);
}

